# AMR Analysis training
## Prerequisite
 * blast (known to work with 2.10.1+)
 * samtools (1.11)
 * skesa
 * prokka (1.14.6)
 * mlst (2.19.6)
 * abricate (1.0.1 | Database: vfdb ecoli_vf ecoh card megares resfinder argannot ncbi plasmidfinder)
 * parsnp 
 ### Data
 1. Reference genome: sample/GCF_000005845.2_ASM584v2_genomic.fna
 2. Sample to analysis: Download [here](https://drive.google.com/drive/folders/1P7_un6I2HzBkObztqbBEffM19ESG2w-u?usp=sharing) and move to samples/ folder 

Download IGV

In [1]:
%%bash
wget https://data.broadinstitute.org/igv/projects/downloads/2.14/IGV_Linux_2.14.1_WithJava.zip -O IGV.zip
unzip IGV.zip

Archive:  IGV.zip


--2022-09-22 22:38:06--  https://data.broadinstitute.org/igv/projects/downloads/2.14/IGV_Linux_2.14.1_WithJava.zip
Resolving data.broadinstitute.org (data.broadinstitute.org)... 69.173.92.29
Connecting to data.broadinstitute.org (data.broadinstitute.org)|69.173.92.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71472977 (68M) [application/zip]
Saving to: ‘IGV.zip’

     0K .......... .......... .......... .......... ..........  0% 31.0M 2s
    50K .......... .......... .......... .......... ..........  0% 81.5K 7m9s
   100K .......... .......... .......... .......... ..........  0%  161K 7m10s
   150K .......... .......... .......... .......... ..........  0%  165K 7m8s
   200K .......... .......... .......... .......... ..........  0%  163K 7m7s
   250K .......... .......... .......... .......... ..........  0% 51.4M 5m56s
   300K .......... .......... .......... .......... ..........  0%  163K 6m6s
   350K .......... .......... .......... .......... .....

CalledProcessError: Command 'b'wget https://data.broadinstitute.org/igv/projects/downloads/2.14/IGV_Linux_2.14.1_WithJava.zip -O IGV.zip\nunzip IGV.zip\n'' returned non-zero exit status 1.

In [1]:
import os, sys
import numpy as np
import pandas as pd

if not os.path.exists('output'):
    os.makedirs('output')

## Pipeline
### Assembly

Create assembly file with skesa

In [ ]:
%%bash
skesa --memory 8 --cores 4 --reads  samples/samples_AS00000197_file-0.fastq.gz samples/samples_AS00000197_file-1.fastq.gz >output/contigs.fasta


Assembly file is stored in output/contigs.fasta

In [ ]:
%%bash
head output/contigs.fasta

### Sequence typing
Using mslt and pubmlst

In [ ]:
%%bash
mlst output/contigs.fasta

### Annotation genome
Using prokka to annotate genome

In [ ]:
%%bash
prokka --force --addgenes --prefix sample --locus sample --outdir output output/contigs.fasta

### Detect AMR genes
Using abricate with ncbi database to find AMR genes

In [ ]:
%%bash
abricate --quiet  --nopath --db ncbi output/contigs.fasta > output/sample_resistome.tsv

In [ ]:
head output/sample_resistome.tsv

### Variant calling
Alignment with reference genome

In [ ]:
%%bash
bwa index samples/GCF_000005845.2_ASM584v2_genomic.fna.gz
bwa mem samples/GCF_000005845.2_ASM584v2_genomic.fna samples/samples_AS00000197_file-0.fastq.gz samples/samples_AS00000197_file-1.fastq.gz  > output/aln-pe.sam


In [ ]:
%%bash
head output/aln-pe.sam

Variant calling with bcftools

In [ ]:
%%bash
samtools view -S -b output/aln-pe.sam > output/aln-pe.bam
samtools sort  output/aln-pe.bam > output/aln-sorted.bam
samtools index output/aln-sorted.bam output/aln-sorted.bam.bai
bcftools mpileup -f samples/GCF_000005845.2_ASM584v2_genomic.fna output/aln-sorted.bam | bcftools call -mv  -Ov > output/variants.vcf

In [ ]:
%%bash
head output/variants.vcf

### Put all together with IGV viewer
Using IGV to display all ouput files

In [ ]:
%%bash
./IGV_Linux_2.14.1/igv.sh

### Phylogeny
Make phylogeny tree with parsnp

In [ ]:
%%bash
parsnp -r samples/GCF_000005845.2_ASM584v2_genomic.fna -d samples/*.fna output/contigs.fasta -o output -c

In [ ]:
from Bio import Phylo
tree = Phylo.read('output/parsnp.tree', "newick")
tree.ladderize()
Phylo.draw(tree)